In [1]:
pip install transformers pillow torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df=pd.read_csv('/kaggle/input/20th-aug-images/updated_dataset.csv')

In [3]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Sl No.,text,Humor,Mechanism,Article link,image_url,labels,texts,...,partial_matching_images,pages_with_matching_images,crop_hints,Headline_Response,Combined_Entities_Response,First Part,Split_Headline_Responses,label,6_way_label,path
0,0,2856,NaN,craigslist ad get paid to attend harvard for me,NaN,NaN,NaN,https://external-preview.redd.it/jt6tIxka4W2mj...,NaN,NaN,...,NaN,NaN,NaN,?\n\nSatire often uses irony and exaggeration ...,.\n\n**Related Keywords:**\n\n- Affiliate mark...,craigslist ad get paid,to promote your affiliate products,0,0.0,/content/drive/MyDrive/20th_aug_capstone/image...
1,1,2093,NaN,this cat chillin on an arm rest,NaN,NaN,NaN,https://preview.redd.it/ygvtnvtoqb831.jpg?widt...,NaN,NaN,...,NaN,NaN,NaN,", looking nonchalant and aloof.\n\n**""He's the...",.\n\nThese are all cat breeds.\n\nThese are al...,this cat chillin,' with its human,0,0.0,/content/drive/MyDrive/20th_aug_capstone/image...
2,2,4974,4974.0,Disney Edits Controversial Mandalorian Scene S...,1.0,GP,https://babylonbee.com/news/disney-edits-contr...,https://media.babylonbee.com/articles/article-...,"Nose, Skin, Head, Organ, Human, Toy, Mammal, G...",NaN,...,https://pbs.twimg.com/media/Em-vOW3XcAQYlGT?fo...,https://www.yucelgumruk.com.tr/?w=why-does-the...,"[x: 139\n, x: 730\n, x: 730\ny: 787\n, x: 139\...",This caption is mocking Disney's recent adapta...,", Baby\n\n**Related Keywords:**\n\n- Baby Yoda...",Disney Edits Controversial Mandalorian Scene So,Far\n\nDisney has faced controversy for,1,NaN,/content/drive/MyDrive/20th_aug_capstone/image...
3,3,10121,10123.0,Hero: This Man Refused To Turn To His Neighbor...,1.0,RL,https://babylonbee.com/news/hero-man-refused-t...,https://media.babylonbee.com/articles/article-...,"Forehead, Nose, Cheek, Lip, Chin, Hairstyle, E...","A, A",...,https://get.pxhere.com/photo/man-person-hair-m...,https://www.elastflexespumas.com.br/how-to-tak...,"[x: 174\n, x: 469\n, x: 469\ny: 393\n, x: 174\...",.\n\nSatire often uses irony and exaggeration ...,", Artist, Light, Composition, Background, Fram...",Hero: This Man Refused To Turn,"Away From His Dreams, Captured In",1,NaN,/content/drive/MyDrive/20th_aug_capstone/image...
4,4,10483,NaN,bentley says national leaders should look to a...,NaN,NaN,NaN,https://external-preview.redd.it/cpqJ9WmQhmFRA...,NaN,NaN,...,NaN,NaN,NaN,.\n\nThis caption is mocking the idea that Ala...,**Related Keywords:**\n\n- Public speaking\n- ...,bentley says national leaders should look,to Paul Chan Mo-po for,0,0.0,/content/drive/MyDrive/20th_aug_capstone/image...


In [4]:
filtered_df = df[df['label'] == 1]

In [5]:
df=filtered_df[['text','path','Humor','Mechanism']]

In [6]:
df.head()

,text,path,Humor,Mechanism
2,Disney Edits Controversial Mandalorian Scene S...,/content/drive/MyDrive/20th_aug_capstone/image...,1.0,GP
3,Hero: This Man Refused To Turn To His Neighbor...,/content/drive/MyDrive/20th_aug_capstone/image...,1.0,RL
6,There Is No Need To Worry About China's Space ...,/content/drive/MyDrive/20th_aug_capstone/image...,0.0,NaN
9,Prime Minister Issues Friendly Reminder To UK ...,/content/drive/MyDrive/20th_aug_capstone/image...,1.0,RL
10,Democrats Wearing Binkies To Wean Themselves O...,/content/drive/MyDrive/20th_aug_capstone/image...,1.0,RL


In [7]:
df['path'] = df['path'].str.replace('/content/drive/MyDrive/20th_aug_capstone','/kaggle/input/20th-aug-images')

/tmp/ipykernel_23/1324731424.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['path'] = df['path'].str.replace('/content/drive/MyDrive/20th_aug_capstone','/kaggle/input/20th-aug-images')


In [8]:
df.head()
df.to_csv('output.csv')

In [9]:
import pandas as pd
from transformers import BlipProcessor, BlipForQuestionAnswering
from PIL import Image
import torch
from tqdm import tqdm
import logging

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Initialize device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the correct BLIP model for question answering
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

# List of questions
questions = [
    "Is there a misdirection in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a contradiction in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a conflict in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a juxtaposition in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a shift in meaning in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a shift in context in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a sudden shift in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a violation of norms in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a violation of expectation in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there an absurdity in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there irony in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a pause in terms of delivery in this image-text combination? Answer only with 'yes', 'no', or 'maybe'.",
    "Is there a cognitive effect in this image-text combination? Answer only with 'yes', 'no', or 'maybe'."
]


def answer_questions(image_path, text):
    """
    Generate answers for a single image-text pair
    """
    try:
        # Load and process image
        image = Image.open(image_path).convert('RGB')
        
        # Initialize list to store answers
        answers = []
        
        # Process each question
        for question in questions:
            # Combine the text with the question
            full_question = f"Context: {text}. {question}"
            
            # Prepare inputs
            inputs = processor(image, full_question, return_tensors="pt").to(device)
            
            # Generate output
            with torch.no_grad():
                outputs = model.generate(**inputs)
            
            # Decode answer
            answer = processor.decode(outputs[0], skip_special_tokens=True)
            answers.append(answer.lower())  # Convert to lowercase for consistency
            
        return answers
        
    except Exception as e:
        logging.error(f"Error processing {image_path}: {str(e)}")
        return ["error"] * len(questions)

def process_csv(csv_path):
    """
    Process the entire CSV file
    """
    try:
        # Read CSV file
        df = pd.read_csv(csv_path)
        logging.info(f"Loaded CSV with {len(df)} rows")
        
        # Initialize columns for answers
        for i, question in enumerate(questions):
            df[f'answer_{i+1}'] = None
            
        # Process each row with progress bar
        for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing images"):
            try:
                # Generate answers for the image-text pair
                answers = answer_questions(row['path'], row['text'])
                
                # Store answers in DataFrame
                for i, answer in enumerate(answers):
                    df.at[index, f'answer_{i+1}'] = answer
                    
            except Exception as e:
                logging.error(f"Error processing row {index}: {str(e)}")
                continue
                
        # Save results
        output_path = 'analysis_results.csv'
        df.to_csv(output_path, index=False)
        logging.info(f"Results saved to {output_path}")
        
        return df
        
    except Exception as e:
        logging.error(f"Error processing CSV: {str(e)}")
        raise

def main():
    csv_path = "/kaggle/working/output.csv"  # Adjust path as needed
    
    try:
        results_df = process_csv(csv_path)
        
        # Print summary statistics
        print("\nAnalysis Summary:")
        for i in range(len(questions)):
            col = f'answer_{i+1}'
            value_counts = results_df[col].value_counts()
            print(f"\nQuestion {i+1} responses:")
            print(value_counts)
            
    except Exception as e:
        logging.error(f"Main execution failed: {str(e)}")

if __name__ == "__main__":
    main()

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

Processing images:   0%|          | 0/10887 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Processing images: 100%|██████████| 10887/10887 [4:01:55<00:00,  1.33s/it]


Analysis Summary:

Question 1 responses:
answer_1
yes          8194
no           1331
same          672
1              43
all            30
             ... 
nerd            1
he is bad       1
wmdt            1
none            1
god ' s         1
Name: count, Length: 222, dtype: int64

Question 2 responses:
answer_2
yes              6876
no               2674
same              393
same thing         96
copyright          90
                 ... 
less co2            1
helicopter          1
all but 1           1
first               1
player number       1
Name: count, Length: 270, dtype: int64

Question 3 responses:
answer_3
yes              6832
no               1885
same             1286
copyright          60
no idea            52
                 ... 
death               1
' no '              1
hate                1
real estate         1
international       1
Name: count, Length: 256, dtype: int64

Question 4 responses:
answer_4
yes          7623
same         1203
no           1104
